In [11]:
import torch
import torch.nn as nn
import torchvision
from torchinfo import summary 
from scripts import data_acquisition, data_setup, engine, plot_loss_curves
from pathlib import Path

In [36]:
DATA_PATH = Path("data")
IMAGE_PATH = Path("pizza_steak_sushi_20")
BATCH_SIZE = 32
EPCOHS = 10
device = "cuda" if torch.cuda.is_available() else "cpu"

In [37]:
data_acquisition.acquire_data(image_path=IMAGE_PATH,
                              url = "https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi_20_percent.zip",
                              zip_name  = "pizza_steak_sushi_20_percent.zip")

Image directory already exists
Done!


## Comparing 2 models

For this section, we will use both the efficientnet and ViT models to train the same dataset and compare the results. We will use the same dataset and the same training parameters for both models. We will also use the same training and validation data for both models. We will then compare the results of both models to see which one performs better.

### Efficientnet

In [38]:
efficientnet_weights = torchvision.models.EfficientNet_B2_Weights.DEFAULT
effnet_transforms = efficientnet_weights.transforms()
efficientnet_model = torchvision.models.efficientnet_b2(weights=efficientnet_weights)
for param in efficientnet_model.parameters():
    param.requires_grad = False


In [39]:
summary(model = efficientnet_model, 
        input_size = (32, 3, 224, 224),
        col_names = ["input_size", "output_size", "num_params", "trainable"],)

Layer (type:depth-idx)                                  Input Shape               Output Shape              Param #                   Trainable
EfficientNet                                            [32, 3, 224, 224]         [32, 1000]                --                        False
├─Sequential: 1-1                                       [32, 3, 224, 224]         [32, 1408, 7, 7]          --                        False
│    └─Conv2dNormActivation: 2-1                        [32, 3, 224, 224]         [32, 32, 112, 112]        --                        False
│    │    └─Conv2d: 3-1                                 [32, 3, 224, 224]         [32, 32, 112, 112]        (864)                     False
│    │    └─BatchNorm2d: 3-2                            [32, 32, 112, 112]        [32, 32, 112, 112]        (64)                      False
│    │    └─SiLU: 3-3                                   [32, 32, 112, 112]        [32, 32, 112, 112]        --                        --
│    └─Sequential: 

In [40]:
efficientnet_model.classifier = nn.Sequential(nn.Dropout(p=0.3, inplace=False),
                                              nn.Linear(in_features=1408, out_features=3, bias=True),
                                              )    

In [41]:
summary(model = efficientnet_model, 
        input_size = (32, 3, 224, 224),
        col_names = ["input_size", "output_size", "num_params", "trainable"],)

Layer (type:depth-idx)                                  Input Shape               Output Shape              Param #                   Trainable
EfficientNet                                            [32, 3, 224, 224]         [32, 3]                   --                        Partial
├─Sequential: 1-1                                       [32, 3, 224, 224]         [32, 1408, 7, 7]          --                        False
│    └─Conv2dNormActivation: 2-1                        [32, 3, 224, 224]         [32, 32, 112, 112]        --                        False
│    │    └─Conv2d: 3-1                                 [32, 3, 224, 224]         [32, 32, 112, 112]        (864)                     False
│    │    └─BatchNorm2d: 3-2                            [32, 32, 112, 112]        [32, 32, 112, 112]        (64)                      False
│    │    └─SiLU: 3-3                                   [32, 32, 112, 112]        [32, 32, 112, 112]        --                        --
│    └─Sequential

In [42]:
effnet_traindataloader, effnet_testdataloader = data_setup.create_dataloaders(
    data_dir = DATA_PATH/IMAGE_PATH,
    batch_size = BATCH_SIZE,
    train_transform = effnet_transforms,
    test_transform = effnet_transforms,
    num_workers = 0
)

In [43]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(efficientnet_model.parameters(), lr=0.001)

In [44]:
history , allpreds, alllabels, effnetwriter = engine.train(model = efficientnet_model,
                                                           data_name = "pizza_steak_sushi",
                                                           train_dataloader = effnet_traindataloader,
                                                           val_dataloader= effnet_testdataloader,
                                                           epochs = EPCOHS,
                                                           model_name = "efficientnet_b2",
                                                           loss_fn = loss_fn,
                                                           optimizer = optimizer,
                                                           device = device
                                                        )

 10%|█         | 1/10 [00:04<00:38,  4.27s/it]

Epoch 1 of 10, Train Loss: 0.9523, Train Acc: 0.6062, Val Loss: 0.7461, Val Acc: 0.8858


 20%|██        | 2/10 [00:08<00:33,  4.18s/it]

Epoch 2 of 10, Train Loss: 0.7169, Train Acc: 0.8187, Val Loss: 0.6006, Val Acc: 0.9472


 30%|███       | 3/10 [00:12<00:29,  4.19s/it]

...

Epoch 8 of 10, Train Loss: 0.3558, Train Acc: 0.8896, Val Loss: 0.3123, Val Acc: 0.9534


 90%|█████████ | 9/10 [00:41<00:04,  4.95s/it]

Epoch 9 of 10, Train Loss: 0.3025, Train Acc: 0.9187, Val Loss: 0.2994, Val Acc: 0.9443


100%|██████████| 10/10 [00:46<00:00,  4.68s/it]

Epoch 10 of 10, Train Loss: 0.2882, Train Acc: 0.9354, Val Loss: 0.2909, Val Acc: 0.9534
